In [ ]:
!pip install -e .. > /dev/null 2>&1

In [ ]:
import os
import gdown
import shutil

from astrohack.gdown_utils import gdown_data
from astrohack.gdown_utils import build_folder_structure

In [ ]:
# Build folder structure & download 

base_name = 'ea25_cal_small_'

datafolder = 'data'
resultsfolder = 'results'

build_folder_structure(datafolder, resultsfolder)

gdown_data(ms_name='ea25_cal_small_before_fixed.split.ms', download_folder=datafolder)
gdown_data(ms_name='ea25_cal_small_after_fixed.split.ms', download_folder=datafolder)

In [ ]:
from astrohack.astrohack_client import astrohack_local_client

log_parms = {'log_level':'DEBUG'}

client = astrohack_local_client(cores=2, memory_limit='8GB', log_parms=log_parms)

In [ ]:
def test_holog_obs_dictionary(holog_obs_dict):
    import os
    import json
    import copy
    
    import numpy as np

    from astrohack._utils._tools import _jsonify

    ref_holog_obj = {}
    ref_holog_obj = copy.deepcopy(holog_obs_dict)

    _jsonify(ref_holog_obj)

    with open(".holog_obs_dict.json") as json_file:
        holog_obj = json.load(json_file)
                          
    assert holog_obj == ref_holog_obj, "Error: holog_obs_descrition dictionary has changes unexpectedly."
    
def test_holog_diagnostics(cell_size, grid_size, number_of_digits=7):
    import json
    
    with open(".holog_diagnostic.json") as json_file:
        json_data = json.load(json_file)
        
    json_data['cell_size'] = [round(x, number_of_digits) for x in json_data['cell_size']]
        
    assert (json_data['cell_size'] == cell_size).all(), "Unexpected change in cell_size occured."
    assert (json_data['grid_size'] == grid_size).all(), "Unexpected change in grid_size occured."

    
def test_center_pixel(file, reference_center_pixels):
    from astrohack.dio import open_image
    
    mds = open_image(file)['ant_ea25']['ddi_0']
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]    
    
    aperture_center_pixels = mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2]
    beam_center_pixels = mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2]
    
    assert np.all(reference_center_pixels['aperture'] == aperture_center_pixels), "There has been a shift in aperture center pixel value(s)"
    assert np.all(reference_center_pixels['beam'] == beam_center_pixels), "There has been a shift in beam center pixel value(s)"

In [ ]:
from astrohack.extract_holog import extract_holog
import numpy as np

beforems = datafolder+'/'+ base_name + 'before_fixed.split.ms'
beforeholog = resultsfolder+'/before.split.holog.zarr'
afterms =  datafolder+'/'+ base_name + 'after_fixed.split.ms'
afterholog = resultsfolder+'/after.split.holog.zarr'

holog_obs_dict = {
    'ddi_0': {
        'map_0': {
            'ant': {
                'ea06': np.array(['ea04']),
                'ea25': np.array(['ea04'])
            },
            'scans': np.array([
                8, 9, 10, 12, 13, 14, 16, 17, 18, 23, 24, 
                25, 27, 28, 29, 31, 32, 33, 38, 39, 40, 
                42, 43, 44, 46, 47, 48, 53, 54, 55, 57
            ])
        }
    }
}


extract_holog(
    ms_name=beforems, 
    holog_name=beforeholog, 
    ddi_sel=[0],
    data_col='CORRECTED_DATA',
    parallel=False,
    overwrite=True,
    reuse_point_zarr=False
)

test_holog_obs_dictionary(holog_obs_dict)

holog_obs_dict = {
    'ddi_0': {
        'map_0': {
            'ant': {
                'ea06': np.array(['ea04']),
                'ea25': np.array(['ea04'])
            },
            'scans': np.array([
                8,  9, 10, 12, 13, 14, 16, 17, 18, 
                23, 24, 25, 27, 28, 29, 31, 32, 33, 
                38, 39, 40, 42, 43, 44, 46, 47, 48, 
                53, 54, 55, 57
            ])
        }
    },
    'ddi_1': {
        'map_0': {
            'ant': {
                'ea06': np.array(['ea04']),
                'ea25': np.array(['ea04'])
            },
            'scans': np.array([
                8,  9, 10, 12, 13, 14, 16, 17, 18, 
                23, 24, 25, 27, 28, 29, 31, 32, 33, 
                38, 39, 40, 42, 43, 44, 46, 47, 48, 
                53, 54, 55, 57
            ])
        }
    }
}


holog_mds_after, _ = extract_holog(
    ms_name=afterms, 
    holog_name=afterholog,
    data_col='CORRECTED_DATA',
    parallel=False,
    overwrite=True,
    reuse_point_zarr=False
)

test_holog_obs_dictionary(holog_obs_dict)


In [ ]:
holog_mds_after

In [ ]:
reference_before_dict = {
    'aperture': np.array([
        [
            [ 
                0.225725131776915+0.12054326352090801j, 
                0.016838151657196862-0.011171227001989446j, 
                0.048663223170791664-0.024197325651061738j, 
                -0.09340400975688747-0.045923630603716084j
            ]
        ]
    ]),
    'beam': np.array([
        [
            [ 
                0.993988815582417+0.10948166283475885j,
                0.005367471552903289+0.01583907120584613j,
                0.014425807274417096+0.0059018780911659395j,
                -0.004021223235578464+0.00040825140567029433j
            ]
        ]
    ])
}

reference_after_dict = {
    'aperture': np.array([
        [
            [ 
                0.12300077664655817-0.06757802469840296j,
                -0.062129865145786264+0.15643752202705213j,
                0.012563562045357843+0.1010680231104541j,
                0.016046659386512153-0.008623951102220646j
            ]
        ]
    ]),
    'beam': np.array([
        [
            [ 
                0.9979274427989393+0.06434919509030099j,
                0.00894231045479574+0.022065573193509103j,
                0.02131330222465877+0.0051265326714060675j,
                0.00271048218206843-0.005807650531399505j
            ]
        ]
    ])
}

In [ ]:
import numpy as np
from astrohack import holog

grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
scan_average = True

holog(
    holog_name=beforeholog, 
    padding_factor=50, 
    grid_interpolation_mode=grid_interpolation_mode,
    chan_average = chan_average,
    reference_scaling_frequency=reference_scaling_frequency,
    scan_average = scan_average,
    overwrite=True,
    phase_fit=True,
    apply_mask=True,
    to_stokes=True,
    parallel=True
)
    
test_center_pixel(file="results/before.split.image.zarr", reference_center_pixels=reference_before_dict)

test_holog_diagnostics(
    cell_size = np.array([-0.0006442, 0.0006442]),
    grid_size = np.array([31, 31]),
    number_of_digits=7
)

holog(
    holog_name=afterholog, 
    padding_factor=50, 
    grid_interpolation_mode=grid_interpolation_mode,
    chan_average = chan_average,
    reference_scaling_frequency=reference_scaling_frequency,
    scan_average = scan_average,
    overwrite=True,
    phase_fit=True,
    apply_mask=True,
    to_stokes=True,
    parallel=True
)

test_center_pixel(file="results/after.split.image.zarr", reference_center_pixels=reference_after_dict)

test_holog_diagnostics(
    cell_size = np.array([-0.0006442, 0.0006442]),
    grid_size = np.array([31, 31]),
    number_of_digits=7
)

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import scipy
import matplotlib.patches as patches

from scipy import constants

from astrohack.dio import open_image

beforeimage = resultsfolder+'/before.split.image.zarr'
afterimage = resultsfolder+'/after.split.image.zarr'

plt.close('all')

chan = 0

ds = open_image("results/before.split.image.zarr")['ant_ea25']['ddi_0']

wavelength = scipy.constants.speed_of_light/ds.chan.values[chan]
l = ds.l.values
m = ds.m.values
u = ds.u.values*wavelength
v = ds.v.values*wavelength

print(u.min(),u.max())

plt.figure()
plt.imshow(np.abs(ds.BEAM[0,chan,0,:,:]),extent=[l.min(), l.max(), m.min(), m.max()])
plt.colorbar()
plt.show()

circle = patches.Circle((0,0), 12.5, fill=False, color='white', alpha=0.7, linewidth=2)

fig, ax =plt.subplots()
plt.imshow(np.abs(ds.APERTURE[0,chan,0,:,:]),extent=[u.min(), u.max(), v.min(), v.max()])

plt.show()

ds

In [ ]:
from astrohack.panel import panel

panel_model = 'rigid'

before_panel = panel(
    image_name=beforeimage, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)
after_panel = panel(
    image_name=afterimage, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)

In [ ]:
import numpy as np
from astrohack.dio import open_panel

def relative_difference(mean, expected):  
    return 2*np.abs(mean - expected)/(abs(mean) + abs(expected))

def verify_panel_shifts(
    panel_list=['3-4', '5-27', '5-37', '5-38'], 
    expected_shift=np.array([-100, 75, 0, 150]),
    ref_mean_shift = np.array([-112.2325, 73.225, -1.455, 139.04  ]),
    antenna='ant_ea25',
    ddi='ddi_0'
):
    
    M_TO_MILS = 39370.1
    
    before_mds = open_panel('results/before.split.panel.zarr')
    after_mds = open_panel('results/after.split.panel.zarr')
    
    before_shift = before_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
    after_shift = after_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
    
    difference = after_shift - before_shift
    
    mean_shift = np.mean(difference, axis=1)
    
    delta_mean_shift = np.abs(mean_shift - expected_shift)
    delta_ref_shift = np.abs(ref_mean_shift - expected_shift)
        
    delta_shift = delta_mean_shift - delta_ref_shift  # New corrections - old corrections --> delta if delta < 0 ==> we improved.
    relative_shift = relative_difference(delta_mean_shift, delta_ref_shift)
       
    if np.any(relative_shift > 1e-6): 
        print("There were changes!")
        for i, delta in enumerate(delta_shift):
            if delta < 0:
                print("{panel}, improved by {delta} mils".format(panel=panel_list[i], delta=delta))
            else:
                print("{panel}, got worse by {delta} mils".format(panel=panel_list[i], delta=delta))

In [ ]:
verify_panel_shifts()